In [75]:
include("./perov_functions.jl")
using Plots
using BenchmarkTools
using Distributions

In [241]:
n=4;nk=100;
system_original=make_symstem(n);
system=copy(system_original);

In [85]:
function e1(system)
    axis=rand(Uniform(-1.,1.),3)
    theta=rand(Uniform(0,2π))
    pos=rand(1:size(system)[1],3)
    s,s1=get_rotation_energy_1(system,1,0,pos,axis,theta);
    if s<0
        system=s1
        end;
    e=get_total_energy_mag(system,1,1/150)
    return e
    end;

In [238]:
function run_sum(n,system)
    e=[]
    a1=[]
    a2=[]
    a3=[]
    for i in 1:n
        axis=rand(Uniform(-1.,1.),3)
        theta=rand(Uniform(0,2π))
        pos=rand(1:size(system)[1],3)
        s,s1=get_rotation_energy_1(system,1,1/150,pos,axis,theta);
        if s<0
            system=s1
            end;
        end
#     vals=get_total_energy_mag(system,1,1/150)
#     energy=vals[1]
#     mag=vals[2]
#     angles=[rad2deg(acos(dot(mag,i))) for i in [[1,0,0],[0,1,0],[0,0,1]]]
#     push!(e,energy)
#     push!(a1,angles[1])
#     push!(a2,angles[2])
#     push!(a3,angles[3])
#     return e,a1,a2,a3
    return system
    end;

In [213]:
@time e,a1,a2,a3=run_sum(500,system);

 26.097600 seconds (288.76 M allocations: 31.006 GiB, 20.48% gc time)


In [248]:
@time sys=run_sum(3000,sys);

  1.439180 seconds (15.33 M allocations: 1.688 GiB, 22.42% gc time)


In [249]:
get_total_energy_mag(sys,1,1/150)

(-1.0454959907732073, [-0.007078498093361179, 0.013858341361780692, -0.012480455713685177])

In [242]:
get_total_energy_mag(system,1,1/150)

(1.507510109282827, [-0.5770239610076704, -0.5532537562435476, -0.6007941657717921])

In [214]:
pl1=plot(e,label="energy")
pl2=plot(a1,label="a1")
plot!(pl2,a2,label="a2")
plot!(pl2,a3,label="a3")
plot(pl1,pl2)

<!DOCTYPE html>
 
 
 Plots.jl

In [211]:
axis=rand(Uniform(-1.,1.),3)
theta=rand(Uniform(0,2π))
pos=rand(1:size(system)[1],3)
get_rotation_energy_1(system,1,1/150,pos,axis,theta)[1]

-0.831146672788021

In [250]:
Plots.plotly()
pl=plot()
for i in sys
    plot_tetra(i,pl)
    end;
display(pl)

<!DOCTYPE html>
 
 
 Plots.jl